In [1]:
import urllib
from urllib.request import urlopen
import pandas as pd

file = urllib.request.urlopen('https://alexip-ml.s3.amazonaws.com/stackexchange_812k.csv.gz')
df = pd.read_csv(file, compression='gzip', header=0, sep=',', quotechar='"')

In [2]:
df.tail(10)

,post_id,parent_id,comment_id,text,category
812122,279993,NaN,536604.0,@Calimo i just plotted the results of each con...,comment
812123,279993,NaN,536608.0,"Ok, but then that's not a ROC curve, which sho...",comment
812124,279993,NaN,536612.0,@Calimo yes i understand. but therein lies my ...,comment
812125,279993,NaN,536451.0,You really want a cost function...,comment
812126,279993,NaN,536454.0,"@Calimo it would seem so, stumbled upon `What ...",comment
812127,279994,NaN,536471.0,"It does run, and gives very valid looking esti...",comment
812128,279998,NaN,536439.0,It seems to me that you are correct; the doubl...,comment
812129,279998,NaN,536514.0,It wouldn't be the first time a grader has mis...,comment
812130,279999,NaN,536802.0,The basic idea is to compare the clustering co...,comment
812131,279999,NaN,542550.0,"As per your other question, your data does not...",comment


In [3]:
import re

# Remove HTML tags
df.text = df.text.str.replace('<[^<]+?>', '')

# Remove Latex
df.text = df.text.str.replace('\\textbf{([^}]*)}', '')

# Remove rows with empty text
filter = df["text"] != ""
df = df[filter]

# Remove 100 words or longer
df = df[df['text'].str.split().str.len().lt(100)]

# Remove 5 words or less
df = df[df['text'].str.split().str.len().gt(5)]

In [4]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

# from: https://stackoverflow.com/a/54398984/773263
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


df['cleanText']=df['text'].map(lambda s:preprocess(s)) 

In [5]:
df.head(15)

,post_id,parent_id,comment_id,text,category,cleanText
2,3,NaN,NaN,What are some valuable Statistical Analysis op...,title,valuable statistical analysis open source proj...
3,4,NaN,NaN,Assessing the significance of differences in d...,title,assessing significance differences distributions
4,6,NaN,NaN,The Two Cultures: statistics vs. machine learn...,title,two cultures statistics machine learning
6,8,NaN,NaN,So how many staticians *does* it take to screw...,title,many staticians take screw lightbulb
7,10,NaN,NaN,Under what conditions should Likert scales be ...,title,conditions likert scales used ordinal interval...
10,22,NaN,NaN,Bayesian and frequentist reasoning in plain En...,title,bayesian frequentist reasoning plain english
11,23,NaN,NaN,Finding the PDF given the CDF,title,finding pdf given cdf
12,25,NaN,NaN,Tools for modeling financial time series,title,tools modeling financial time series
15,31,NaN,NaN,What is the meaning of p values and t values i...,title,meaning values values statistical tests
17,36,NaN,NaN,Examples for teaching: Correlation does not me...,title,examples teaching correlation mean causation


In [6]:
df.to_csv("clean_data.csv", sep='\t')